In [199]:
import pandas as pd # 0.23.0
import numpy as np
import xml.etree.ElementTree as ET
import csv

In [200]:
def get_delimiter(file_path: str) -> str:
    with open(file_path, 'r') as csvfile:
        delimiter = str(csv.Sniffer().sniff(csvfile.read()).delimiter)
        return delimiter

In [201]:
fileName = 'combinations.csv'
df = pd.read_csv(fileName, sep = get_delimiter(fileName), on_bad_lines='skip')

In [202]:
tmpNull = 12345
df.iloc[:, 5:10] = df.iloc[:, 5:10].fillna(tmpNull).astype(int).replace(tmpNull, '-')
df = df.replace(np.nan, '-')
print("Are there any NaN values: ", df.isna().any().any())


Are there any NaN values:  False


In [203]:
df.iloc[:, 5:10] = df.iloc[:, 5:10].astype(str)
df

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode,Bemerkung
0,C:\TESTDATENGENERATOR\Temp_NoHybrid_Bifuel.xml,BFE230703TEST0001,N1,Empty,B,10,40,-,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
1,C:\TESTDATENGENERATOR\Temp_OVC_Bifuel.xml,BFE230703TEST0002,N1,OVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
2,C:\TESTDATENGENERATOR\Temp_NOVC_Bifuel.xml,BFE230703TEST0003,N1,NOVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
3,C:\TESTDATENGENERATOR\Temp_NoHybrid_Bifuel.xml,BFE230703TEST0004,N1,Empty,B,10,40,-,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
4,C:\TESTDATENGENERATOR\Temp_OVC_Bifuel.xml,BFE230703TEST0005,N1,OVC-HEV,B,10,40,0,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
...,...,...,...,...,...,...,...,...,...,...,...
58,C:\TESTDATENGENERATOR\Temp_NOVC_Monofuel.xml,BFE230703TEST0059,N1,NOVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"
59,C:\TESTDATENGENERATOR\Temp_OVC_Monofuel.xml,BFE230703TEST0060,N1,OVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
60,C:\TESTDATENGENERATOR\Temp_NOVC_Monofuel.xml,BFE230703TEST0061,N1,NOVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
61,C:\TESTDATENGENERATOR\Temp_OVC_Monofuel.xml,BFE230703TEST0062,M1,OVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"


---------------------------------------------------------------------------

In [204]:
df['Info manueller Change in DB: vehicleNatureCode '].unique()

array(['-', '38'], dtype=object)

Map values


In [205]:
dict_VCC = {'N1': '01', 'M1': '30'} # 35 and 38 need to be mapped
dict_CHVC = {'Empty': 'empty', 'OVC-HEV': 'vchev',
             'NOVC-HEV': 'nchev', 'OVC-FCHV': 'cfchv',
             'NOVC-FCHV': 'nfchv'}

In [212]:
def createVIN(row):

  emptyElement = '$'

  newVIN = 'BFE'

  # check for N1
  vcc = row['#VehicleCategoryCode']

  if vcc == 'N1':

    com = row['Info manueller Change in DB: vehicleNatureCode ']

    if com  != '-':
      newVIN = newVIN + com + dict_CHVC[row['ClassOfHybridVehicleCode']]
    else:
      newVIN = newVIN + dict_VCC[vcc] + dict_CHVC[row['ClassOfHybridVehicleCode']]
  else:
    newVIN = newVIN + dict_VCC[vcc] + dict_CHVC[row['ClassOfHybridVehicleCode']]

  fuelTypeCode = str(row['#FuelTypeCode'])

  if fuelTypeCode == '-':
    newVin = newVIN + emptyElement
  else:
    newVin = newVIN + fuelTypeCode

  for col in ['#FuelCode1','#FuelCode2', '#FuelCode3']:

    val = str(row[col])

    if len(val) < 2:
      if val == '-':
        val = emptyElement + emptyElement
      else:
        val = val + emptyElement

    newVIN = newVIN + val

  return newVIN

In [213]:
df['#VIN'] = df.apply(lambda row: createVIN(row), axis=1)
df

,Template,#VIN,#VehicleCategoryCode,ClassOfHybridVehicleCode,#FuelTypeCode,#FuelCode1,#FuelCode2,#FuelCode3,FuelCode4,Info manueller Change in DB: vehicleNatureCode,Bemerkung
0,C:\TESTDATENGENERATOR\Temp_NoHybrid_Bifuel.xml,BFE01empty1040$$,N1,Empty,B,10,40,-,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
1,C:\TESTDATENGENERATOR\Temp_OVC_Bifuel.xml,BFE01vchev10400$,N1,OVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
2,C:\TESTDATENGENERATOR\Temp_NOVC_Bifuel.xml,BFE01nchev10400$,N1,NOVC-HEV,B,10,40,0,-,-,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
3,C:\TESTDATENGENERATOR\Temp_NoHybrid_Bifuel.xml,BFE38empty1040$$,N1,Empty,B,10,40,-,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
4,C:\TESTDATENGENERATOR\Temp_OVC_Bifuel.xml,BFE38vchev10400$,N1,OVC-HEV,B,10,40,0,-,38,"BI-Fuel, tstof Y, Erdgas (CNG) / Benzin"
...,...,...,...,...,...,...,...,...,...,...,...
58,C:\TESTDATENGENERATOR\Temp_NOVC_Monofuel.xml,BFE01nchev20$$0$,N1,NOVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"
59,C:\TESTDATENGENERATOR\Temp_OVC_Monofuel.xml,BFE38vchev20$$0$,N1,OVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
60,C:\TESTDATENGENERATOR\Temp_NOVC_Monofuel.xml,BFE38nchev20$$0$,N1,NOVC-HEV,M,20,-,0,-,38,"Mono-Fuel, Diesel/ elektrisch, tstof F"
61,C:\TESTDATENGENERATOR\Temp_OVC_Monofuel.xml,BFE30vchev20$$0$,M1,OVC-HEV,M,20,-,0,-,-,"Mono-Fuel, Diesel/ elektrisch, tstof F"


In [214]:
for i in df['#VIN'].unique():
  if '-' in i:
    print(i)

------------------------------------------------------------

In [215]:
df['Template'] = df.apply(lambda row: row['Template'].replace("C:\\TESTDATENGENERATOR\\", "templates/"), axis=1)
print(df.head())
df.to_csv('combinations_modified.csv')

                             Template              #VIN #VehicleCategoryCode  \
0  templates/Temp_NoHybrid_Bifuel.xml  BFE01empty1040$$                   N1   
1       templates/Temp_OVC_Bifuel.xml  BFE01vchev10400$                   N1   
2      templates/Temp_NOVC_Bifuel.xml  BFE01nchev10400$                   N1   
3  templates/Temp_NoHybrid_Bifuel.xml  BFE38empty1040$$                   N1   
4       templates/Temp_OVC_Bifuel.xml  BFE38vchev10400$                   N1   

  ClassOfHybridVehicleCode #FuelTypeCode #FuelCode1 #FuelCode2 #FuelCode3  \
0                    Empty             B         10         40          -   
1                  OVC-HEV             B         10         40          0   
2                 NOVC-HEV             B         10         40          0   
3                    Empty             B         10         40          -   
4                  OVC-HEV             B         10         40          0   

  FuelCode4 Info manueller Change in DB: vehicleNatureCo

In [216]:
valuesToReplace = [col for col in df.columns if col.startswith('#')]
valuesToReplace

['#VIN',
 '#VehicleCategoryCode',
 '#FuelTypeCode',
 '#FuelCode1',
 '#FuelCode2',
 '#FuelCode3']

In [217]:
summe = 0
l = 0
i = 1
for template in df['Template'].unique():

  print(i, ": ", template)

  # all the attribute combinations which need to be with current template
  tmpDF = df[df.Template == template]

  summe = summe + len(tmpDF)

  for index, row in tmpDF.iterrows():

    # load template
    tree = ET.parse(template)
    root = tree.getroot()

    for att in valuesToReplace:

      for element in root.iter():
        #if element.text and element.text.startswith(att):
        if element.text == att:
          #if att == '-':
          #  print("empty")
          element.text = row[att]

    # save modified XML file
    l = l + 1
    tree.write('outputs/' + row['#VIN'] + '.xml', encoding='utf-8', xml_declaration=True)

  i += 1


1 :  templates/Temp_NoHybrid_Bifuel.xml
2 :  templates/Temp_OVC_Bifuel.xml
3 :  templates/Temp_NOVC_Bifuel.xml
4 :  templates/Temp_NoHybrid_Trifuel.xml
5 :  templates/Temp_OVC_Trifuel.xml
6 :  templates/Temp_NOVC_Trifuel.xml
7 :  templates/Temp_NoHybrid_Monofuel.xml
8 :  templates/Temp_OVC_Monofuel.xml
9 :  templates/Temp_NOVC_Monofuel.xml
10 :  templates/Temp_OVC-FCHV.xml
11 :  templates/Temp_NOVC-FCHV.xml
12 :  templates/Temp_REINELEKTRISCH_E.xml
13 :  templates/Temp_Temp_WASSERSTOFF_W.xml


In [218]:
print(summe)

63


In [219]:
print(l)

63
